In [1]:
import argparse
import collections
import datetime
import functools
import json
import logging
import os
import pickle
import random
from typing import Any, List, Tuple, TypeVar

import haiku as hk
import jax
import jax.numpy as jnp
import jmp
import msgpack
import numpy as np
import pandas as pd
import optax
import sklearn.metrics

import femr.datasets
import femr.extension.dataloader
import femr.models.dataloader
import femr.models.transformer

T = TypeVar("T")

In [2]:
batch_info_path = 'EHRSHOT_ASSETS/features/clmbr_batches/batch_info.msgpack'
loader_config_path = 'EHRSHOT_ASSETS/features/clmbr_batches/loader_config.msgpack'
batches_path = 'EHRSHOT_ASSETS/features/clmbr_batches/'
data_path = 'EHRSHOT_ASSETS/femr/extract/'
model_dir = 'EHRSHOT_ASSETS/models/clmbr/clmbr_model/'

In [3]:
with open(batch_info_path, "rb") as data_file:
    byte_data = data_file.read()

with open(loader_config_path, "rb") as loader_file:
    byte_loader = loader_file.read()

data_loaded = msgpack.unpackb(byte_data)
loader_loaded = msgpack.unpackb(byte_loader)


with open(os.path.join(model_dir, "config.msgpack"), "rb") as f:
    config = msgpack.load(f, use_list=False)
random.seed(config["seed"])

In [4]:
loader_loaded['transformer']['dictionary']['regular']
len(loader_loaded['transformer']['dictionary']['regular'])

1729229

In [5]:
print(data_loaded.keys())
print(data_loaded['config'].keys())
len(data_loaded['config']['task']['labels'])
data_loaded['config']['transformer']['dictionary']['regular']

dict_keys(['batches', 'config', 'data_path'])
dict_keys(['seed', 'splits', 'task', 'transformer'])


[{'code_string': 'SNOMED/3950001',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.18811663446051943},
 {'code_string': 'Domain/OMOP generated',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.18650920196731693},
 {'code_string': 'Gender/F',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.11572716107139833},
 {'code_string': 'Gender/M',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.10392800444784726},
 {'code_string': 'Visit/OP',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.10238170672430387},
 {'code_string': 'Medicare Specialty/A0',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.08490693944802999},
 {'code_string': 'Ethnicity/Not Hispanic',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.07075057942837912},
 {

In [6]:
# data_loaded['config']['transformer']['dictionary']['regular']
i_list = []
val_list = []
for i in data_loaded['config']['transformer']['dictionary']['regular']:
    if i['text_string']:
        i_list.append(i['text_string'])
    if i['val_start']:
        val_list.append(i['val_start'])
len(i_list), len(data_loaded['config']['transformer']['dictionary']['regular']), len(val_list)

(13492, 1729229, 28711)

In [7]:
data_loaded['config']['transformer'].keys()

dict_keys(['dictionary', 'is_hierarchical', 'max_size', 'min_size', 'vocab_size'])

In [8]:
print(loader_loaded.keys())
print(loader_loaded['transformer'].keys())
print(loader_loaded['transformer']['dictionary'].keys())
len(loader_loaded['transformer']['dictionary']['regular'])
len(loader_loaded['task']['labels'])
loader_loaded['splits']

dict_keys(['transformer', 'task', 'seed', 'splits'])
dict_keys(['vocab_size', 'dictionary', 'min_size', 'max_size', 'is_hierarchical'])
dict_keys(['age_stats', 'regular'])


[['train', 0, 70], ['dev', 70, 85], ['test', 85, 100]]

##### Compute represenetation starts here

In [9]:
# len(batch_info['config']['task']['labels'])

In [10]:
with open(os.path.join(model_dir, "config.msgpack"), "rb") as f:
    config = msgpack.load(f, use_list=False)

    random.seed(config["seed"])

In [11]:
config = hk.data_structures.to_immutable_dict(config)
batch_info_path = os.path.join(batches_path, "batch_info.msgpack")

with open(batch_info_path, "rb") as f:
    batch_info = msgpack.load(f, use_list=False)

In [12]:
patient_labels = collections.defaultdict(list)

for pid, age, label in batch_info["config"]["task"]["labels"]:
    patient_labels[pid].append((age, label))

In [13]:
loader = femr.extension.dataloader.BatchLoader(data_path, batch_info_path)

When mapping codes, dropped 4736 out of 39811


In [14]:
# data_loaded['batches']['dev']
# loader
# patient_labels

In [15]:
def model_fn(config, batch):
    model = femr.models.transformer.EHRTransformer(config)(batch, no_task=True)
    return model

dummy_batch = loader.get_batch("train", 0)
# dummy_batch = jax.tree_map(lambda a: jnp.array(a), dummy_batch)

In [16]:
dummy_batch

{'num_patients': 4,
 'num_indices': 1334,
 'patient_ids': array([115972796, 115969885, 115967185, ...,         0,         0,
                0], dtype=uint64),
 'offsets': array([0, 0, 0, ..., 0, 0, 0], dtype=uint32),
 'transformer': {'length': 32768,
  'tokens': array([ 0,  3, 21, ...,  0,  0,  0], dtype=uint32),
  'valid_tokens': array([ True,  True,  True, ..., False, False, False]),
  'ages': array([0.        , 0.99930555, 0.99930555, ..., 0.        , 0.        ,
         0.        ], dtype=float32),
  'normalized_ages': array([-1.3069556, -1.3069555, -1.3069555, ...,  0.       ,  0.       ,
          0.       ], dtype=float32),
  'label_indices': array([     3,     64,    224, ..., 131072, 131072, 131072], dtype=uint32)},
 'task': {'label_ages': array([25282078, 26578846, 27317457, ...,        0,        0,        0],
        dtype=uint32),
  'labels': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}}

In [17]:
dummy_batch['transformer']['normalized_ages'][[3, 64, 224]]

array([-1.3069555, -1.3056272, -1.3055892], dtype=float32)

In [18]:
dummy_batch_0 = jax.tree_map(lambda a: jnp.array(a), dummy_batch)
dummy_batch

{'num_patients': 4,
 'num_indices': 1334,
 'patient_ids': array([115972796, 115969885, 115967185, ...,         0,         0,
                0], dtype=uint64),
 'offsets': array([0, 0, 0, ..., 0, 0, 0], dtype=uint32),
 'transformer': {'length': 32768,
  'tokens': array([ 0,  3, 21, ...,  0,  0,  0], dtype=uint32),
  'valid_tokens': array([ True,  True,  True, ..., False, False, False]),
  'ages': array([0.        , 0.99930555, 0.99930555, ..., 0.        , 0.        ,
         0.        ], dtype=float32),
  'normalized_ages': array([-1.3069556, -1.3069555, -1.3069555, ...,  0.       ,  0.       ,
          0.       ], dtype=float32),
  'label_indices': array([     3,     64,    224, ..., 131072, 131072, 131072], dtype=uint32)},
 'task': {'label_ages': array([25282078, 26578846, 27317457, ...,        0,        0,        0],
        dtype=uint32),
  'labels': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}}

In [19]:
pids = dummy_batch['patient_ids']
pages = dummy_batch['task']['label_ages']
np.sum(pages != 0)
np.unique(pages).shape
# for i in range(200):
#     temp = loader.get_batch("train", i)
print(dummy_batch.keys())
print('num_patients:', dummy_batch['num_patients'])
print('num_indices:', dummy_batch['num_indices'])
print(batch_info['batches']['train'][1])
print('patient_ids:', dummy_batch['patient_ids'][0:dummy_batch['num_patients']])
# dummy_batch
dummy_batch_transformer = dummy_batch['transformer']
print(dummy_batch_transformer['tokens'].shape)
print(dummy_batch_transformer['valid_tokens'].sum())
dummy_batch_transformer
# dummy_batch_task = dummy_batch['task']
# print(dummy_batch_task.keys())

dict_keys(['num_patients', 'num_indices', 'patient_ids', 'offsets', 'transformer', 'task'])
num_patients: 4
num_indices: 1334
(13, ((1881, 0), (1389, 0), (1554, 0), (5437, 0), (4422, 0), (4254, 0), (4682, 0), (38, 0), (1931, 0), (3854, 0), (2583, 0), (5136, 0), (1562, 0), (3457, 0), (840, 0), (640, 0)))
patient_ids: [115972796 115969885 115967185 115970692]
(131072,)
90515


{'length': 32768,
 'tokens': array([ 0,  3, 21, ...,  0,  0,  0], dtype=uint32),
 'valid_tokens': array([ True,  True,  True, ..., False, False, False]),
 'ages': array([0.        , 0.99930555, 0.99930555, ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 'normalized_ages': array([-1.3069556, -1.3069555, -1.3069555, ...,  0.       ,  0.       ,
         0.       ], dtype=float32),
 'label_indices': array([     3,     64,    224, ..., 131072, 131072, 131072], dtype=uint32)}

In [20]:
# np.where(dummy_batch_transformer['tokens'] != 0)
# for i in range(1, len(dummy_batch_transformer['tokens'])):
#     t_0 = dummy_batch_transformer['tokens'][i-1]
#     t_1 = dummy_batch_transformer['tokens'][i]
#     if t_0 == 0 and t_1 != 0:
#         print(i-1, i)
# print('')

# for i in range(1, len(dummy_batch_transformer['tokens'])):
#     t_0 = dummy_batch_transformer['tokens'][i-1]
#     t_1 = dummy_batch_transformer['tokens'][i]
#     if t_0 != 0 and t_1 == 0:
#         print(i-1, i)

# for i in range(1, len(dummy_batch_transformer['valid_tokens'])):
#     t_0 = dummy_batch_transformer['valid_tokens'][i-1]
#     t_1 = dummy_batch_transformer['valid_tokens'][i]
#     if t_0 and not t_1:
#         print(i-1, i)

for i in range(1, len(dummy_batch_transformer['ages'])):
    t_0 = dummy_batch_transformer['ages'][i-1]
    t_1 = dummy_batch_transformer['ages'][i]
    if t_0 != 0 and t_1 == 0:
        print(i-1, i)
dummy_batch_transformer['label_indices'][1333:1340]

21532 21533
52377 52378
83657 83658
129553 129554


array([129325, 131072, 131072, 131072, 131072, 131072, 131072],
      dtype=uint32)

In [21]:
dummy_batch_transformer['label_indices'][1334]

131072

In [22]:
dummy_batch_transformer

{'length': 32768,
 'tokens': array([ 0,  3, 21, ...,  0,  0,  0], dtype=uint32),
 'valid_tokens': array([ True,  True,  True, ..., False, False, False]),
 'ages': array([0.        , 0.99930555, 0.99930555, ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 'normalized_ages': array([-1.3069556, -1.3069555, -1.3069555, ...,  0.       ,  0.       ,
         0.       ], dtype=float32),
 'label_indices': array([     3,     64,    224, ..., 131072, 131072, 131072], dtype=uint32)}

In [23]:
# dummy_batch_transformer['tokens'][129554:129600]
# dummy_batch_transformer['tokens'][21533:21640]
for i in range(1, len(dummy_batch['task']['label_ages'])):
    t_0 = dummy_batch['task']['label_ages'][i-1]
    t_1 = dummy_batch['task']['label_ages'][i]
    if t_0 != 0 and t_1 == 0:
        print(i-1, i)

1333 1334


In [24]:
with open('EHRSHOT_ASSETS/features/clmbr_features.pkl', 'rb') as f:
    df = pickle.load(f)
df['data_matrix'].shape

(406379, 768)

In [25]:
np.unique(df['label_values'])

array([False])

In [26]:
print(dummy_batch.keys())
dummy_batch

dict_keys(['num_patients', 'num_indices', 'patient_ids', 'offsets', 'transformer', 'task'])


{'num_patients': 4,
 'num_indices': 1334,
 'patient_ids': array([115972796, 115969885, 115967185, ...,         0,         0,
                0], dtype=uint64),
 'offsets': array([0, 0, 0, ..., 0, 0, 0], dtype=uint32),
 'transformer': {'length': 32768,
  'tokens': array([ 0,  3, 21, ...,  0,  0,  0], dtype=uint32),
  'valid_tokens': array([ True,  True,  True, ..., False, False, False]),
  'ages': array([0.        , 0.99930555, 0.99930555, ..., 0.        , 0.        ,
         0.        ], dtype=float32),
  'normalized_ages': array([-1.3069556, -1.3069555, -1.3069555, ...,  0.       ,  0.       ,
          0.       ], dtype=float32),
  'label_indices': array([     3,     64,    224, ..., 131072, 131072, 131072], dtype=uint32)},
 'task': {'label_ages': array([25282078, 26578846, 27317457, ...,        0,        0,        0],
        dtype=uint32),
  'labels': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}}

In [27]:
# dummy_batch["transformer"]["label_indices"]
# dummy_batch["transformer"]["length"]
config

FlatMap({
  'data_path': '/share/pi/nigam/data/som-rit-phi-starr-prod.starr_omop_cdm5_deid_2023_02_08_extract_v8',
  'batch_info_path': '/share/pi/nigam/rthapa84/data/clmbr_text_assets/clmbr_lr_0.0001_wd_0.0_id_0.0_td_0.0_rt_per_head_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_tasks_8192_val_size_70_obs/clmbr_batches/batch_info.msgpack',
  'seed': 97,
  'task': FlatMap({'type': 'clmbr', 'vocab_size': 8192}),
  'transformer': FlatMap({
                   'vocab_size': 65536,
                   'hidden_size': 768,
                   'intermediate_size': 3072,
                   'n_heads': 12,
                   'n_layers': 12,
                   'rotary': 'per_head',
                   'attention_width': 496,
                   'internal_dropout': 0,
                   'is_hierarchical': False,
                   'note_embedding_data': None,
                 }),
  'learning_rate': 0.0001,
  'max_grad_norm': 1.0,
  'weight_decay': 0.0,
  'n_epochs': 100,
})

In [28]:


rng = jax.random.PRNGKey(42)
model = hk.transform(model_fn)

In [29]:
loader.get_number_of_batches('train'), loader.get_number_of_batches('dev'), loader.get_number_of_batches('test')

(183, 45, 45)

In [30]:
with open(os.path.join(model_dir, "best"), "rb") as f:
    params = pickle.load(f)

@functools.partial(jax.jit, static_argnames="config")
def compute_repr(params, rng, config, batch):
    return model.apply(params, rng, config, batch)

database = femr.datasets.PatientDatabase(data_path)

results = collections.defaultdict(list)


In [31]:
# for split in ("train", "dev", "test"):
split = 'train'
dev_index = 1
# for dev_index in range(loader.get_number_of_batches(split)):
raw_batch = loader.get_batch(split, dev_index)
batch = jax.tree_map(lambda a: jnp.array(a), raw_batch)

repr, mask = compute_repr(
        femr.models.transformer.convert_params(params, dtype=jnp.float16),
        rng,
        config,
        batch,
    )

repr = np.array(repr)

p_index = batch["transformer"]["label_indices"] // batch["transformer"]["length"]

for i in range(batch["num_indices"]):
    r = repr[i, :]

    label_pid = raw_batch["patient_ids"][p_index[i]]
    label_age = raw_batch["task"]["label_ages"][i]

    offset = raw_batch["offsets"][p_index[i]]
    results[label_pid].append((label_age, offset, r))


Compiling the transformer ... (131072,) (4096,)
WITHOUT AGE


In [32]:
batch

{'num_indices': Array(1962, dtype=int32, weak_type=True),
 'num_patients': Array(16, dtype=int32, weak_type=True),
 'offsets': Array([0, 0, 0, ..., 0, 0, 0], dtype=uint32),
 'patient_ids': Array([115968530, 115968693, 115968648, ...,         0,         0,
                0], dtype=uint32),
 'task': {'label_ages': Array([38625961, 38867867, 38868220, ...,        0,        0,        0],      dtype=uint32),
  'labels': Array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)},
 'transformer': {'ages': Array([0.        , 0.99930555, 0.99930555, ..., 0.        , 0.        ,
         0.        ], dtype=float32),
  'label_indices': Array([    15,    140,    217, ..., 131072, 131072, 131072], dtype=uint32),
  'length': Array(8192, dtype=int32, weak_type=True),
  'normalized_ages': Array([-1.3069556, -1.3069555, -1.3069555, ...,  0.       ,  0.       ,
          0.       ], dtype=float32),
  'tokens': Array([0, 3, 7, ..., 0, 0, 0], dtype=uint32),
  'valid_tokens': Array([ True,  True,  True, ..., Fa

In [33]:
sum_indices = 0
for split in ("train", "dev", "test"):
    for dev_index in range(loader.get_number_of_batches(split)):
        raw_batch = loader.get_batch(split, dev_index)
        sum_indices += raw_batch["num_indices"]

In [34]:
for key in results.keys():
    print(len(results[key]))

154
73
209
78
52
109
52
99
80
375
99
82
241
78
70
111


In [35]:
raw_batch['transformer']['valid_tokens'].shape

(131072,)

In [36]:
batch

{'num_indices': Array(1962, dtype=int32, weak_type=True),
 'num_patients': Array(16, dtype=int32, weak_type=True),
 'offsets': Array([0, 0, 0, ..., 0, 0, 0], dtype=uint32),
 'patient_ids': Array([115968530, 115968693, 115968648, ...,         0,         0,
                0], dtype=uint32),
 'task': {'label_ages': Array([38625961, 38867867, 38868220, ...,        0,        0,        0],      dtype=uint32),
  'labels': Array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)},
 'transformer': {'ages': Array([0.        , 0.99930555, 0.99930555, ..., 0.        , 0.        ,
         0.        ], dtype=float32),
  'label_indices': Array([    15,    140,    217, ..., 131072, 131072, 131072], dtype=uint32),
  'length': Array(8192, dtype=int32, weak_type=True),
  'normalized_ages': Array([-1.3069556, -1.3069555, -1.3069555, ...,  0.       ,  0.       ,
          0.       ], dtype=float32),
  'tokens': Array([0, 3, 7, ..., 0, 0, 0], dtype=uint32),
  'valid_tokens': Array([ True,  True,  True, ..., Fa

In [37]:
raw_batch

{'num_patients': 32,
 'num_indices': 1209,
 'patient_ids': array([115969870, 115973250, 115970792, ...,         0,         0,
                0], dtype=uint64),
 'offsets': array([0, 0, 0, ..., 0, 0, 0], dtype=uint32),
 'transformer': {'length': 4096,
  'tokens': array([0, 3, 7, ..., 0, 0, 0], dtype=uint32),
  'valid_tokens': array([ True,  True,  True, ..., False, False, False]),
  'ages': array([0.        , 0.99930555, 0.99930555, ..., 0.        , 0.        ,
         0.        ], dtype=float32),
  'normalized_ages': array([-1.3069556, -1.3069555, -1.3069555, ...,  0.       ,  0.       ,
          0.       ], dtype=float32),
  'label_indices': array([     3,     20,     39, ..., 131072, 131072, 131072], dtype=uint32)},
 'task': {'label_ages': array([33466629, 33467869, 33905609, ...,        0,        0,        0],
        dtype=uint32),
  'labels': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}}

In [38]:
batch

{'num_indices': Array(1962, dtype=int32, weak_type=True),
 'num_patients': Array(16, dtype=int32, weak_type=True),
 'offsets': Array([0, 0, 0, ..., 0, 0, 0], dtype=uint32),
 'patient_ids': Array([115968530, 115968693, 115968648, ...,         0,         0,
                0], dtype=uint32),
 'task': {'label_ages': Array([38625961, 38867867, 38868220, ...,        0,        0,        0],      dtype=uint32),
  'labels': Array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)},
 'transformer': {'ages': Array([0.        , 0.99930555, 0.99930555, ..., 0.        , 0.        ,
         0.        ], dtype=float32),
  'label_indices': Array([    15,    140,    217, ..., 131072, 131072, 131072], dtype=uint32),
  'length': Array(8192, dtype=int32, weak_type=True),
  'normalized_ages': Array([-1.3069556, -1.3069555, -1.3069555, ...,  0.       ,  0.       ,
          0.       ], dtype=float32),
  'tokens': Array([0, 3, 7, ..., 0, 0, 0], dtype=uint32),
  'valid_tokens': Array([ True,  True,  True, ..., Fa

In [62]:
batch['transformer']['tokens'].shape
# 131072 / 32
raw_batch

{'num_patients': 32,
 'num_indices': 1209,
 'patient_ids': array([115969870, 115973250, 115970792, ...,         0,         0,
                0], dtype=uint64),
 'offsets': array([0, 0, 0, ..., 0, 0, 0], dtype=uint32),
 'transformer': {'length': 4096,
  'tokens': array([0, 3, 7, ..., 0, 0, 0], dtype=uint32),
  'valid_tokens': array([ True,  True,  True, ..., False, False, False]),
  'ages': array([0.        , 0.99930555, 0.99930555, ..., 0.        , 0.        ,
         0.        ], dtype=float32),
  'normalized_ages': array([-1.3069556, -1.3069555, -1.3069555, ...,  0.       ,  0.       ,
          0.       ], dtype=float32),
  'label_indices': array([     3,     20,     39, ..., 131072, 131072, 131072], dtype=uint32)},
 'task': {'label_ages': array([33466629, 33467869, 33905609, ...,        0,        0,        0],
        dtype=uint32),
  'labels': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}}

In [66]:
t = raw_batch['transformer']['tokens']
l = raw_batch['transformer']['label_indices']
l_1 = l[l != 131072]

In [78]:
a = raw_batch['transformer']['ages']
a[l_1]

array([9.9930555e-01, 2.3240715e+04, 2.3545541e+04, ..., 2.1560477e+04,
       2.1580469e+04, 2.1597000e+04], dtype=float32)

In [40]:
loader_loaded['transformer']['dictionary']['regular']

[{'code_string': 'SNOMED/3950001',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.18811663446051943},
 {'code_string': 'Domain/OMOP generated',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.18650920196731693},
 {'code_string': 'Gender/F',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.11572716107139833},
 {'code_string': 'Gender/M',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.10392800444784726},
 {'code_string': 'Visit/OP',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.10238170672430387},
 {'code_string': 'Medicare Specialty/A0',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.08490693944802999},
 {'code_string': 'Ethnicity/Not Hispanic',
  'text_string': '',
  'type': 0,
  'val_end': 0.0,
  'val_start': 0.0,
  'weight': -0.07075057942837912},
 {

In [41]:
train_batches = data_loaded['batches']['train']

In [42]:
batch_info['config']

{'seed': 97,
 'splits': (('train', 0, 70), ('dev', 70, 85), ('test', 85, 100)),
 'task': {'labeler_type': 'boolean',
  'labels': ((115973769, 29657479.0, False),
   (115973769, 29663114.0, False),
   (115973769, 29663294.0, False),
   (115973769, 29663424.0, False),
   (115973769, 29663999.0, False),
   (115973769, 29664179.0, False),
   (115973769, 29667169.0, False),
   (115973769, 29668319.0, False),
   (115973769, 30130004.0, False),
   (115973769, 31218682.0, False),
   (115973769, 32334499.0, False),
   (115973769, 34121543.0, False),
   (115973769, 34753547.0, False),
   (115973769, 35179835.0, False),
   (115973769, 35443099.0, False),
   (115973769, 35459230.0, False),
   (115973769, 35481035.0, False),
   (115973427, 28236090.0, False),
   (115973427, 28472287.0, False),
   (115973427, 28613361.0, False),
   (115973427, 28689799.0, False),
   (115973427, 28934509.0, False),
   (115973427, 29517683.0, False),
   (115973427, 29752396.0, False),
   (115973427, 29942442.0, False)